In [9]:
%reload_ext autoreload
%autoreload 3

import torch
from torch import nn
import pytorch_lightning as pl
from src.seq2conn.simple_vae import SimpleVAE
from src.seq2conn.data import RandomDataModule
from src.seq2conn.models.dgmg import DGMG

[autoreload of src.seq2conn.models.dgmg failed: Traceback (most recent call last):
  File "/home/morteza/micromamba/envs/seq2conn/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 271, in check
    superreload(m, reload, self.old_objects, self.shell)
  File "/home/morteza/micromamba/envs/seq2conn/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 496, in superreload
    update_generic(old_obj, new_obj)
  File "/home/morteza/micromamba/envs/seq2conn/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 393, in update_generic
    update(a, b)
  File "/home/morteza/micromamba/envs/seq2conn/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 361, in update_class
    update_instances(old, new)
  File "/home/morteza/micromamba/envs/seq2conn/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 319, in update_instances
    object.__setattr__(ref, "__class__", new)
TypeError: __class__ assignment only supported

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f'Using {device}...')

Using cuda...


In [11]:
DGMG(.2, 10, 5)

DGMG(
  (graph_embed): GraphEmbed(
    (node_gating): Sequential(
      (0): Linear(in_features=10, out_features=1, bias=True)
      (1): Sigmoid()
    )
    (node_to_graph): Linear(in_features=10, out_features=20, bias=True)
  )
  (graph_prop): GraphProp(
    (message_funcs): ModuleList(
      (0-4): 5 x Linear(in_features=21, out_features=20, bias=True)
    )
    (node_update_funcs): ModuleList(
      (0-4): 5 x GRUCell(20, 10)
    )
  )
  (add_node_agent): AddNode(
    (add_node): Linear(in_features=20, out_features=1, bias=True)
    (node_type_embed): Embedding(1, 10)
    (initialize_hv): Linear(in_features=30, out_features=10, bias=True)
  )
  (add_edge_agent): AddEdge(
    (add_edge): Linear(in_features=30, out_features=1, bias=True)
  )
  (choose_dest_agent): ChooseDestAndUpdate(
    (choose_dest): Linear(in_features=20, out_features=1, bias=True)
  )
)

In [3]:
batch_size = 5
n_timesteps = 10
input_dim = 15
hidden_dim = 20
n_layers = n_timesteps

input = torch.randn(batch_size, n_timesteps, input_dim)

encoder = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True)

decoder = nn.LSTM(hidden_dim, input_dim, n_layers, batch_first=True)

out_enc, (h_enc, cell) = encoder(input)
h_enc = h_enc.permute(1, 0, 2)
out_dec, (h_dec, cell) = decoder(h_enc)
h_dec = h_dec.permute(1, 0, 2)

input.shape, h_enc.shape, out_enc.shape, h_dec.shape, out_dec.shape

# input.shape: (batch_size, n_timesteps, input_dim)
# h1.shape: (batch_size, n_layers, hidden_dim)
# out1.shape: (batch_size, n_timesteps, hidden_dim)
# h2.shape: (batch_size, n_layers, input_dim)
# out2.shape: (batch_size, n_timesteps, input_dim)


(torch.Size([5, 10, 15]),
 torch.Size([5, 10, 20]),
 torch.Size([5, 10, 20]),
 torch.Size([5, 10, 15]),
 torch.Size([5, 10, 15]))

In [ ]:
datamodule = RandomDataModule(batch_size=batch_size, n_timesteps=n_timesteps, input_dim=input_dim)
model = SimpleVAE(input_dim, hidden_dim, n_layers)

trainer = pl.Trainer(max_epochs=100, log_every_n_steps=1)

trainer.fit(model, datamodule=datamodule)